In [1]:
from ultralytics import YOLO
import json
import cv2
import numpy as np

In [2]:
# Uncomment below lines to train the pretrained yolo on custom dataset
# model=YOLO('yolov9c-seg.pt')
# model.train(data='./dataset/data.yaml', epochs=5, imgsz=640) 

In [5]:
# load the best model
custom_model = YOLO('best_v8m.pt')

In [6]:
custom_model.info()

YOLOv8m-seg summary: 331 layers, 27240227 parameters, 0 gradients


(331, 27240227, 0, 0.0)

In [7]:
# validate the trained custom model 
val_results = custom_model.val(data='./dataset/data.yaml')

Ultralytics YOLOv8.2.24  Python-3.12.3 torch-2.3.0+cpu CPU (Intel Core(TM) i5-8265U 1.60GHz)
YOLOv8m-seg summary (fused): 245 layers, 27222963 parameters, 0 gradients


val: Scanning C:\Users\Kishan.Ajudiya\Desktop\thesis\Lawn_Segmentation\dataset\val\labels.cache... 9 images, 0 backgrounds, 0 corrupt: 100%|██████████| 9/9 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:21<00:00, 21.17s/it]


                   all          9         10      0.818        0.9      0.902      0.838      0.818        0.9      0.902      0.812
Speed: 5.8ms preprocess, 2313.7ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs\segment\val7


In [8]:
print(val_results)


ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000001C73FEA2120>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.0410

In [26]:
# test = custom_model.val(data='./dataset/data.yaml', task='test')
# print(test)
# Set the test flag to True
test_results = custom_model.predict(source='dataset/test_random/913bf910dbc12ebb96311c5920944e76.jpg', save=True)
print(test_results)



image 1/1 c:\Users\Kishan.Ajudiya\Desktop\thesis\Lawn_Segmentation\dataset\test_random\913bf910dbc12ebb96311c5920944e76.jpg: 480x640 1 grass, 1906.8ms
Speed: 4.0ms preprocess, 1906.8ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs\segment\predict4
[ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: ultralytics.engine.results.Masks object
names: {0: 'grass'}
obb: None
orig_img: array([[[113, 132,  89],
        [116, 135,  92],
        [120, 139, 100],
        ...,
        [ 91, 119,  89],
        [ 86, 111,  83],
        [ 82, 107,  79]],

       [[120, 137,  94],
        [119, 138,  95],
        [123, 141, 102],
        ...,
        [ 97, 123,  93],
        [ 84, 109,  81],
        [ 71,  96,  68]],

       [[117, 131,  89],
        [115, 132,  89],
        [117, 135,  96],
        ...,
        [ 97, 121,  89],
        [107, 131, 101],
        [112, 136, 106]],

    

In [23]:
# Code from ultralytics
import cv2
import supervision as sv

image = cv2.imread("dataset/test_random/913bf910dbc12ebb96311c5920944e76.jpg")
model = YOLO('best_v8m.pt')
result = model(image)[0]
detections = sv.Detections.from_ultralytics(result)
print(detections)



0: 480x640 1 grass, 1349.7ms
Speed: 7.2ms preprocess, 1349.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)
Detections(xyxy=array([[    0.55415,      157.35,      734.45,       438.7]], dtype=float32), mask=array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]]]), confidence=array([    0.54563], dtype=float32), class_id=array([0]), tracker_id=None, data={'class_name': array(['grass'], dtype='<U5')})


In [24]:
predictions = []
for box, score, cls, mask in zip(detections.xyxy, detections.confidence, detections.class_id, detections.mask):
    prediction = {
        "bounding_box": box.tolist(),  # Convert the bounding box to a list
        "score": float(score),  # Convert the score to a float
        "class": int(cls),  # Convert the class to an integer
        "mask": mask.tolist() if mask is not None else None  # Convert the mask to a list if it exists
    }
    predictions.append(prediction)

# Save the predictions to a JSON file
output_file = "predictions.json"
with open(output_file, 'w') as f:
    json.dump(predictions, f, indent=4)

print(f"Predictions saved to {output_file}")

Predictions saved to predictions.json


In [27]:
# Load the image
image_path = "dataset/test_random/913bf910dbc12ebb96311c5920944e76.jpg"
image = cv2.imread(image_path)

# Load the predictions from the JSON file
output_file = "predictions.json"
with open(output_file, 'r') as f:
    predictions = json.load(f)

# Define colors for different classes (just an example, you can define more colors as needed)
colors = [
    (0, 255, 255),   # Green for class 0
    (255, 0, 0),   # Blue for class 1
    (0, 0, 255),   # Red for class 2
    # Add more colors for additional classes if needed
]

# Loop through the predictions and draw bounding boxes and masks
for prediction in predictions:
    box = prediction["bounding_box"]
    score = prediction["score"]
    cls = prediction["class"]
    mask = prediction["mask"]
    
    # Draw bounding box
    start_point = (int(box[0]), int(box[1]))
    end_point = (int(box[2]), int(box[3]))
    color = colors[cls % len(colors)]
    thickness = 1
    cv2.rectangle(image, start_point, end_point, color, thickness)
    
    # Draw mask if it exists
    if mask is not None:
        mask = np.array(mask, dtype=np.uint8)
        
        # Create a colored mask
        colored_mask = np.zeros_like(image, dtype=np.uint8)
        colored_mask[mask == 1] = color
        
        # Blend the colored mask with the original image
        alpha = 0.4  # Transparency factor
        image = cv2.addWeighted(image, 1 - alpha, colored_mask, alpha, 0)
    
    # Put the class and score text
    label = f"Class: {cls}, Score: {score:.2f}"
    cv2.putText(image, label, (start_point[0], start_point[1] - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

# Display the resulting image
cv2.imshow("Annotated Image", image)
while True:
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
